In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("Recommendation.csv", dtype={"id": str})

In [3]:
df.head(2).T

,0,1
id,179731375,249221065
name,Balo nữ da dầy thời trang unisex màu sắc tươi ...,Ví da đựng thẻ ngân hàng (7 màu) - 9205
description,Balo nữ da dầy thời trang unisex màu sắc tươ...,...
original_price,14250,10260
brand,OEM,OEM
product_source,Balo & vali,Balo & vali
category,Balo nữ,Root
rating_average,0.0,0.0
current_seller,Thiên Ân Balo,SHOP THỜI TRANG 2N


In [4]:
df["description_01"] = df.description.str.lower().str.strip(" \t\n-_–")

In [5]:
sentences = df["description_01"].values

In [6]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")

In [7]:
vectors = []
for i in range(df.shape[0]):
    v = model.encode(sentences[i])
    vectors.append(v)
    if i % 100 == 0:
        print(i)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18300
18400
18

In [8]:
from sklearn.metrics.pairwise import cosine_similarity

In [9]:
df["vector"] = vectors

In [10]:
t = df[["id", "vector"]]

In [11]:
n = t.shape[0]

In [12]:
n

41581

In [13]:
DF = []
for i in range(n):
    _id = t.loc[i, "id"]
    _v = t.loc[i, "vector"].reshape(1, -1)  # reshape để đúng shape (1, dim)

    tt = t[t["id"] != _id].copy()
    vectors = np.stack(tt["vector"].values)  # shape (n-1, dim)

    tt["sim"] = cosine_similarity(_v, vectors)[0]
    tt = tt.sort_values(by="sim", ascending=False).head(20)[["id", "sim"]].rename(columns={"id": "id_dst"})
    tt["id"] = _id
    DF.append(tt)
    if i % 100 == 0:
        print(i)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18300
18400
18

In [14]:
DF = pd.concat(DF)

In [15]:
DF.shape

(831620, 3)

In [16]:
DF.head()

,id_dst,sim,id
6711,138384287,0.774411,179731375
181,138625265,0.758488,179731375
5738,80699007,0.754257,179731375
6015,258525799,0.753002,179731375
10348,258526013,0.753002,179731375


In [18]:
df01 = DF.merge(df[["id", "name"]])

In [20]:
info = df[["id", "name", "brand", "category"]]
info.columns = ["id_dst", "name_dst", "brand_dst", "category_dst"]
df02 = df01.merge(info)

In [22]:
df03 = df02[["id", "name", "sim", "id_dst", "name_dst", "brand_dst", "category_dst"]]

In [24]:
df03.to_csv("D:/career/master/data_analysis/ck/Advanced_Data_Analyst/data/Product_Recommendation.csv", index=False, encoding="utf-8-sig")

In [31]:
from multiprocessing import Pool, cpu_count

def compute_top20(i):
    _id = t.loc[i, "id"]
    _v = t.loc[i, "vector"].reshape(1, -1)

    tt = t[t["id"] != _id].copy()
    vectors = np.stack(tt["vector"].values)
    tt["sim"] = cosine_similarity(_v, vectors)[0]
    tt = tt.sort_values(by="sim", ascending=False).head(20)[["id", "sim"]].rename(columns={"id": "id_dst"})
    tt["id"] = _id
    return tt

In [ ]:
# Run in parallel
if __name__ == "__main__":
    with Pool(cpu_count()) as p:
        results = list(p.map(compute_top20, range(len(t))))
    
    DF = pd.concat(results, ignore_index=True)
    DF.to_csv("recommended_list.csv", index=False)